In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print os.path.join(dirname, filename)

.\data\items.csv
.\data\items_new.csv
.\data\item_categories.csv
.\data\item_categories_new.csv
.\data\sales_train.csv
.\data\sales_train_ag_m.csv
.\data\sales_train_new.csv
.\data\sample_submission.csv
.\data\shops.csv
.\data\shops_new.csv
.\data\test.csv
.\data\test_new.csv
.\data\X_sparse_test.pkl
.\data\X_sparse_train.pkl
.\data\y_train.pkl


In [3]:
sales_train_ag_m = pd.read_csv('.\data\sales_train_ag_m.csv', index_col=0)
test_new = pd.read_csv('.\data\\test_new.csv', index_col=0)       

In [4]:
sales_train_ag_m.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_price,item_category_id,glob_item_category,city
0,0,2,27,1,2013,1.0,2499.0,19,Игры,Адыгея
42141,0,42,21553,1,2013,1.0,199.0,40,Кино,СПб
42142,0,42,21554,1,2013,1.0,199.0,40,Кино,СПб
42143,0,42,21565,1,2013,3.0,349.0,55,Музыка,СПб
42144,0,42,21571,1,2013,1.0,199.0,55,Музыка,СПб


In [5]:
test_new.head()

,shop_id,item_id,month,year,item_category_id,glob_item_category,city
0,5,5037,11,2015,19,Игры,Вологда
1,5,5320,11,2015,55,Музыка,Вологда
2,5,5233,11,2015,19,Игры,Вологда
3,5,5232,11,2015,23,Игры,Вологда
4,5,5268,11,2015,20,Игры,Вологда


In [46]:
import pickle
with open('.\data\X_sparse_train.pkl', 'rb') as f:
    X_sparse_train = pickle.load(f)
with open('.\data\X_sparse_test.pkl', 'rb') as f:
    X_sparse_test = pickle.load(f)
with open('.\data\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

# Построение моделей обучения

Так как в данной задаче нужно предсказывать колличество проданных товаров (целые числа) в диапазоне от 0 до 20, то будем рассматривать данную задачу, как задачу многоклассовой классификации с 21 классом.

## *Разобьем обучающие данные на обучающую и тестовые данные. Попробуем предсказывать результаты в октябре 2015 года

In [40]:
sales_train_ag_m.shape

(1609123, 10)

In [42]:
sales_train_ag_m[sales_train_ag_m['date_block_num']!=33].shape

(1577592, 10)

In [47]:
X_sparse_train.shape

(1609123L, 22372L)

In [59]:
X_tr = X_sparse_train.tocsc()[:1577592]
X_val = X_sparse_train.tocsc()[1577592:]
y_tr = y_train[:1577592]
y_val = y_train[1577592:]

# 1) Базовое решение

Будем считать, что продажи в ноябре 2015года точно такие же как продажи в октябре 2015 года. Построим Previous_value_benchmark модель.

In [14]:
Previous_value_data = sales_train_ag_m[sales_train_ag_m["date_block_num"]==33][['item_id','shop_id', "item_cnt_month"]]
Previous_value_data.head()

,item_id,shop_id,item_cnt_month
1598611,5657,42,2.0
1598612,5659,42,1.0
1598613,5660,42,6.0
1598614,5661,42,1.0
1598619,5671,42,6.0


In [22]:
Previous_value_data['item_cnt_month'].clip(0,20,inplace=True)
Previous_value_data['item_cnt_month'].value_counts()

1.0     21351
2.0      5071
3.0      1907
4.0       959
5.0       582
6.0       341
20.0      261
7.0       226
8.0       158
9.0       138
10.0      103
11.0       65
12.0       63
0.0        59
13.0       54
15.0       50
14.0       46
16.0       34
18.0       25
17.0       21
19.0       17
Name: item_cnt_month, dtype: int64

In [23]:
test_new.shape

(214200, 7)

In [24]:
Previous_value_pred = pd.merge(test_new, Previous_value_data, how='left', left_on=["shop_id","item_id"], right_on = ["shop_id","item_id"])
print(Previous_value_pred.shape)

(214200, 8)


In [25]:
Previous_value_pred.head()

,shop_id,item_id,month,year,item_category_id,glob_item_category,city,item_cnt_month
0,5,5037,11,2015,19,Игры,Вологда,NaN
1,5,5320,11,2015,55,Музыка,Вологда,NaN
2,5,5233,11,2015,19,Игры,Вологда,1.0
3,5,5232,11,2015,23,Игры,Вологда,NaN
4,5,5268,11,2015,20,Игры,Вологда,NaN


In [20]:
Previous_value_pred['item_cnt_month'].count() #видим, что смогли предсказать всего 28680 товаров из 214200

28680

In [27]:
Previous_value_pred['item_cnt_month'].value_counts()

1.0     19264
2.0      4830
3.0      1806
4.0       880
5.0       505
6.0       311
7.0       199
20.0      197
8.0       136
9.0       115
10.0       74
11.0       56
0.0        53
12.0       52
13.0       47
14.0       40
15.0       35
16.0       28
18.0       21
17.0       19
19.0       12
Name: item_cnt_month, dtype: int64

Заполним остальные значения, которые не смогли предсказать самым распространенным классом т.е единицей

In [28]:
Previous_value_pred.fillna(1,inplace=True)
Previous_value_pred['item_cnt_month'].value_counts()

1.0     204784
2.0       4830
3.0       1806
4.0        880
5.0        505
6.0        311
7.0        199
20.0       197
8.0        136
9.0        115
10.0        74
11.0        56
0.0         53
12.0        52
13.0        47
14.0        40
15.0        35
16.0        28
18.0        21
17.0        19
19.0        12
Name: item_cnt_month, dtype: int64

In [36]:
Previous_value_pred['ID'] = Previous_value_pred.index

In [37]:
Previous_value_pred.head()

,shop_id,item_id,month,year,item_category_id,glob_item_category,city,item_cnt_month,ID
0,5,5037,11,2015,19,Игры,Вологда,1.0,0
1,5,5320,11,2015,55,Музыка,Вологда,1.0,1
2,5,5233,11,2015,19,Игры,Вологда,1.0,2
3,5,5232,11,2015,23,Игры,Вологда,1.0,3
4,5,5268,11,2015,20,Игры,Вологда,1.0,4


In [38]:
Previous_value_pred[['ID','item_cnt_month']].to_csv('submission.csv',index=False) 

#score 1.39, если заполнять пропуски нулями, то оценка 1.16

In [35]:
submission = pd.read_csv('.\data\\sample_submission.csv', index_col=0)  
submission.head()

,item_cnt_month
ID,
0,0.5
1,0.5
2,0.5
3,0.5
4,0.5


Попробуем построить модель, которая будет превосходить качество MSE = 1.16, полученное для Previous_value_benchmark модели

# 2) PCA (снижение размерности данных)

In [70]:
X_tr.shape

(1577592, 22372)

In [72]:
from sklearn.decomposition import TruncatedSVD
model = TruncatedSVD(n_components=100, random_state=42)
model.fit(X_tr)
model.explained_variance_ratio_ #массив, который показывает сколько дисперсии объясняет каждая компонента

array([0.01630939, 0.06696476, 0.06398931, 0.0539004 , 0.04193671,
       0.03674792, 0.02581724, 0.01890698, 0.01679816, 0.01598106,
       0.01578382, 0.01520302, 0.01434516, 0.01415863, 0.0140989 ,
       0.01390913, 0.01371938, 0.01290835, 0.01284216, 0.01260276,
       0.01183955, 0.01174064, 0.01036972, 0.01009546, 0.00977398,
       0.00957145, 0.0089397 , 0.00861004, 0.00836326, 0.00768269,
       0.00765518, 0.00747585, 0.00741647, 0.00737028, 0.00722508,
       0.00677323, 0.00648216, 0.00642854, 0.00608415, 0.00579655,
       0.0057351 , 0.00548333, 0.00542333, 0.00533645, 0.00528025,
       0.00516335, 0.0050527 , 0.00495527, 0.00486297, 0.00477543,
       0.00451981, 0.00441068, 0.00426821, 0.00421267, 0.0038521 ,
       0.00356284, 0.00351258, 0.0034172 , 0.00335697, 0.00329302,
       0.00325701, 0.00321547, 0.00313723, 0.00306551, 0.00304674,
       0.00298734, 0.00298281, 0.00293838, 0.00290254, 0.0027679 ,
       0.00274478, 0.00270295, 0.0025641 , 0.00254786, 0.00253

In [80]:
with open('.\data\model_pca.pkl', 'wb') as f:
    pickle.dump(model,f, protocol=2)

In [73]:
X_tr_svd = model.transform(X_tr)

In [74]:
X_val_svd = model.transform(X_val)

# 3) Логистическая регрессия

In [78]:
from sklearn.linear_model import LogisticRegression
c = 0.1 #параметр регуляризации, т.е насколько сильная регуляризация
log_reg = LogisticRegression(C=c, penalty='l2', solver='lbfgs',class_weight='balanced', multi_class = 'multinomial')
from sklearn.model_selection import TimeSeriesSplit #для данных с временной привязкой
rang = TimeSeriesSplit(n_splits=5, max_train_size=None)

In [ ]:
from sklearn.model_selection import cross_val_score 
score = cross_val_score(estimator=log_reg, X=X_tr_svd, y=y_tr, cv=rang,scoring='neg_mean_squared_error') 
score.mean() #средняя оценка score.std() #разброс по оценкам